In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))

In [ ]:

def split(size, obs):
    x = []
    y = []

    for i in range(len(obs)-SEQUENCE_SIZE):
        window = obs[i:(i+SEQUENCE_SIZE)]
        after_window = obs[i+SEQUENCE_SIZE]
        window = [[x] for x in window]
        x.append(window)
        y.append(after_window)

    return np.array(x),np.array(y)





# **...... BITCOIN......**

In [ ]:
btcpath='/content/BTC-USD (6).csv'


In [ ]:
btc=pd.read_csv(btcpath)

In [ ]:
btc

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-01,963.658020,1003.080017,958.698975,998.325012,998.325012,147775008
1,2017-01-02,998.617004,1031.390015,996.702026,1021.750000,1021.750000,222184992
2,2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966,1043.839966,185168000
3,2017-01-04,1044.400024,1159.420044,1044.400024,1154.729980,1154.729980,344945984
4,2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,1013.380005,510199008
...,...,...,...,...,...,...,...
2474,2023-10-11,27392.076172,27474.115234,26561.099609,26873.320313,26873.320313,13648094333
2475,2023-10-12,26873.292969,26921.439453,26558.320313,26756.798828,26756.798828,9392909295
2476,2023-10-13,26752.878906,27092.697266,26686.322266,26862.375000,26862.375000,15165312851
2477,2023-10-14,26866.203125,26969.000000,26814.585938,26861.707031,26861.707031,5388116782


In [ ]:
btc_close=btc.iloc[: , 4:5].values

In [ ]:
btc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2479 non-null   object 
 1   Open       2479 non-null   float64
 2   High       2479 non-null   float64
 3   Low        2479 non-null   float64
 4   Close      2479 non-null   float64
 5   Adj Close  2479 non-null   float64
 6   Volume     2479 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 135.7+ KB


In [ ]:
btc_close

array([[  998.325012],
       [ 1021.75    ],
       [ 1043.839966],
       ...,
       [26862.375   ],
       [26861.707031],
       [27024.675781]])

In [ ]:
btc_close=scaler.fit_transform(btc_close)

In [ ]:
btc_close

array([[0.00330246],
       [0.00365319],
       [0.00398393],
       ...,
       [0.39055219],
       [0.39054219],
       [0.39298224]])

In [ ]:
# Split Data........

btc_train_size = int(len(btc_close)*.8 )
btc_train=btc_close[:btc_train_size]
btc_test=btc_close[btc_train_size:]

In [ ]:
SEQUENCE_SIZE=30
btcx_train,btcy_train = split(SEQUENCE_SIZE,btc_train)
btcx_test,btcy_test = split(SEQUENCE_SIZE,btc_test)

print("Shape of training set: {}".format(btcx_train.shape))
print("Shape of test set: {}".format(btcx_test.shape))

Shape of training set: (1953, 30, 1, 1)
Shape of test set: (466, 30, 1, 1)


In [ ]:
btcx_train = btcx_train.reshape( btcx_train.shape[0], btcx_train.shape[1], 1)

In [ ]:
btcx_train.shape

(1953, 30, 1)

In [ ]:
btcy_train.shape

(1953, 1)

In [ ]:
model=Sequential()
model.add(LSTM(units=30,return_sequences=True,input_shape=(btcx_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=60,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=90,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=120,return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 30, 30)            3840      
                                                                 
 dropout_4 (Dropout)         (None, 30, 30)            0         
                                                                 
 lstm_5 (LSTM)               (None, 30, 60)            21840     
                                                                 
 dropout_5 (Dropout)         (None, 30, 60)            0         
                                                                 
 lstm_6 (LSTM)               (None, 30, 90)            54360     
                                                                 
 dropout_6 (Dropout)         (None, 30, 90)            0         
                                                                 
 lstm_7 (LSTM)               (None, 120)              

In [ ]:
model.fit(btcx_train,btcy_train,epochs=100,batch_size=32)

Epoch 1/100
62/62 [==============================] - 19s 108ms/step - loss: 0.0127
Epoch 2/100
62/62 [==============================] - 6s 103ms/step - loss: 0.0032
Epoch 3/100
62/62 [==============================] - 6s 89ms/step - loss: 0.0026
Epoch 4/100
62/62 [==============================] - 6s 104ms/step - loss: 0.0029
Epoch 5/100
62/62 [==============================] - 6s 88ms/step - loss: 0.0026
Epoch 6/100
62/62 [==============================] - 7s 107ms/step - loss: 0.0027
Epoch 7/100
62/62 [==============================] - 6s 87ms/step - loss: 0.0023
Epoch 8/100
62/62 [==============================] - 6s 97ms/step - loss: 0.0021
Epoch 9/100
62/62 [==============================] - 6s 100ms/step - loss: 0.0027
Epoch 10/100
62/62 [==============================] - 6s 96ms/step - loss: 0.0021
Epoch 11/100
62/62 [==============================] - 6s 102ms/step - loss: 0.0026
Epoch 12/100
62/62 [==============================] - 6s 91ms/step - loss: 0.0018
Epoch 13/100
62/62

In [ ]:
btcx_test.shape

In [ ]:
btcy_pred=model.predict(btcx_test)

In [ ]:

print('MAE =',mean_absolute_error(btcy_test,btcy_pred))


In [ ]:
print('MAPE =',mean_absolute_percentage_error(btcy_test,btcy_pred))

In [ ]:
print('MSE=',mean_squared_error(btcy_test,btcy_pred))

In [ ]:
#Root Mean Sqquare Erorr
print('RMSE=',np.sqrt(mean_squared_error(btcy_test,btcy_pred)))

# **......LETCOIN......**

In [ ]:
ltcpath='/content/LTC-USD (1).csv'


In [ ]:
ltc=pd.read_csv(ltcpath)

In [ ]:
ltc

In [ ]:
ltc_close=ltc.iloc[: , 4:5].values

In [ ]:
ltc.info()

In [ ]:
ltc_close

In [ ]:
ltc_close=scaler.fit_transform(ltc_close)

In [ ]:
ltc_close

In [ ]:
ltc_train_size = int(len(ltc_close)*.8 )
ltc_train=ltc_close[:ltc_train_size]
ltc_test=ltc_close[ltc_train_size:]

In [ ]:
SEQUENCE_SIZE=30
ltcx_train,ltcy_train = split(SEQUENCE_SIZE,ltc_train)
ltcx_test,ltcy_test = split(SEQUENCE_SIZE,ltc_test)

print("Shape of training set: {}".format(ltcx_train.shape))
print("Shape of test set: {}".format(ltcx_test.shape))

In [ ]:
ltcx_train = ltcx_train.reshape( ltcx_train.shape[0], ltcx_train.shape[1], 1)

In [ ]:
ltcx_train.shape

In [ ]:
model=Sequential()
model.add(LSTM(units=30,return_sequences=True,input_shape=(ltcx_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=60,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=90))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

In [ ]:
model.fit(ltcx_train,ltcy_train,epochs=10,batch_size=30)

In [ ]:
ltcy_pred=model.predict(ltcx_test)

In [ ]:

print('MAE =',mean_absolute_error(ltcy_test,ltcy_pred))


In [ ]:
print('MAPE =',mean_absolute_percentage_error(ltcy_test,ltcy_pred))

In [ ]:
print('MSE=',mean_squared_error(ltcy_test,ltcy_pred))

In [ ]:
#Root Mean Squre Erorr
print('RMSE=',np.sqrt(mean_squared_error(ltcy_test,ltcy_pred)))

# **......ETHEREUM......**

In [ ]:
ethpath='/content/ETH-USD.csv'


In [ ]:
eth=pd.read_csv(ethpath)

In [ ]:
eth

In [ ]:
eth_close=eth.iloc[: , 4:5].values

In [ ]:
eth.info()

In [ ]:
eth_close

In [ ]:

eth_close=scaler.fit_transform(eth_close)

In [ ]:
eth_close

In [ ]:
eth_train_size = int(len(eth_close)*.8 )
eth_train=eth_close[:eth_train_size]
eth_test=eth_close[eth_train_size:]

In [ ]:
SEQUENCE_SIZE=30
ethx_train,ethy_train = split(SEQUENCE_SIZE,eth_test)
ethx_test,ethy_test = split(SEQUENCE_SIZE,eth_test)

print("Shape of training set: {}".format(ethx_train.shape))
print("Shape of test set: {}".format(ethx_test.shape))

In [ ]:
ethx_train = ethx_train.reshape( ethx_train.shape[0], ethx_train.shape[1], 1)

In [ ]:
ethx_train.shape

In [ ]:
model=Sequential()
model.add(LSTM(units=30,return_sequences=True,input_shape=(ethx_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=60,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=90))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

In [ ]:
model.fit(ethx_train,ethy_train,epochs=10,batch_size=30)

In [ ]:
ethy_pred=model.predict(ethx_test)

In [ ]:
print('MAE =',mean_absolute_error(ethy_test,ethy_pred))


In [ ]:
print('MAPE =',mean_absolute_percentage_error(ethy_test,ethy_pred))

In [ ]:
print('MSE=',mean_squared_error(ethy_test,ethy_pred))

In [ ]:
#Root Mean Square Erorre
print('RMSE=',np.sqrt(mean_squared_error(ethy_test,ethy_pred)))

# **......XMR.....**

In [ ]:
xmr_path='/content/XMR-USD.csv'

In [ ]:
xmr=pd.read_csv(xmr_path)

In [ ]:
xmr

In [ ]:
xmr_close=xmr.iloc[: , 4:5].values

In [ ]:
xmr_close

In [ ]:
xmr_close=scaler.fit_transform(xmr_close)

In [ ]:
xmr_close

In [ ]:
xmr_train_size = int(len(xmr_close)*.8 )
xmr_train=xmr_close[:xmr_train_size]
xmr_test=xmr_close[xmr_train_size:]

In [ ]:
SEQUENCE_SIZE=30
xmrx_train,xmry_train = split(SEQUENCE_SIZE,xmr_train)
xmrx_test,xmry_test = split(SEQUENCE_SIZE,xmr_test)

print("Shape of training set: {}".format(xmrx_train.shape))
print("Shape of test set: {}".format(xmrx_test.shape))

In [ ]:
xmrx_train = xmrx_train.reshape( xmrx_train.shape[0], xmrx_train.shape[1], 1)

In [ ]:
xmrx_train.shape

In [ ]:
model=Sequential()
model.add(LSTM(units=30,return_sequences=True,input_shape=(xmrx_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=60,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=90))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

In [ ]:
model.fit(xmrx_train,xmry_train,epochs=10,batch_size=30)

In [ ]:
xmry_pred=model.predict(xmrx_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
print('MAE =',mean_absolute_error(xmry_test,xmry_pred))


In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
print('MAPE =',mean_absolute_percentage_error(xmry_test,xmry_pred))

In [ ]:
from sklearn.metrics import mean_squared_error
print('MSE=',mean_squared_error(xmry_test,xmry_pred))

In [ ]:
#Root Mean Square Erorre
print('RMSE=',np.sqrt(mean_squared_error(xmry_test,xmry_pred)))